In [100]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_score, mean_absolute_error
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn import svm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
import sklearn

np.set_printoptions(precision=4)

In [101]:
data = pd.read_csv('../Processed Data/RaceData/final_df.csv')

display(data)

,season,round,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,driver,grid,podium,...,constructor_minardi,constructor_prost,constructor_red_bull,constructor_renault,constructor_sauber,constructor_team_lotus,constructor_toro_rosso,constructor_toyota,constructor_tyrrell,constructor_williams
0,1983,1,False,False,True,False,False,keke_rosberg,1,15,...,0,0,0,0,0,0,0,0,0,1
1,1983,1,False,False,True,False,False,prost,2,6,...,0,0,0,1,0,0,0,0,0,0
2,1983,1,False,False,True,False,False,tambay,3,4,...,0,0,0,0,0,0,0,0,0,0
3,1983,1,False,False,True,False,False,piquet,4,1,...,0,0,0,0,0,0,0,0,0,0
4,1983,1,False,False,True,False,False,warwick,5,7,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15293,2022,22,True,False,False,False,False,kevin_magnussen,16,17,...,0,0,0,0,0,0,0,0,0,0
15294,2022,22,True,False,False,False,False,gasly,17,14,...,0,0,0,0,0,0,0,0,0,0
15295,2022,22,True,False,False,False,False,bottas,18,15,...,0,0,0,0,0,0,0,0,0,0
15296,2022,22,True,False,False,False,False,albon,19,13,...,0,0,0,0,0,0,0,0,0,1


In [102]:
df = data.copy()

train = df[df.season <2022]
X_train = train.drop(['driver', 'podium'], axis = 1)
y_train = train.podium

scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)

In [103]:
comparison_dict ={'model':[],
                  'params': [],
                  'score': []}

# Implement Model

In [104]:
model = sklearn.linear_model.LogisticRegression()
model.fit(X_train, y_train)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

# Evaluate Model

In [105]:
def score_classification(model):
    score = 0
    for circuit in df[df.season == 2022]['round'].unique():

        test = df[(df.season == 2022) & (df['round'] == circuit)]
        X_test = test.drop(['driver', 'podium'], axis = 1)
        y_test = test.podium

        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

        # make predictions
        #test = np.transpose(np.array(model.predict(X_test, return_std=True)))
        test = model.predict(X_test)
        #prediction_df = pd.DataFrame(test, columns = ['y_mean', 'y_std'])
        prediction_df = pd.DataFrame(test, columns=['y_pred'])
        prediction_df['actual'] = y_test.reset_index(drop = True)

        #prediction_df = prediction_df[prediction_df.y_std < 2]

        #score += mean_absolute_error(prediction_df.actual, prediction_df.y_mean)
        score += mean_absolute_error(prediction_df.actual, prediction_df.y_pred)

    model_score = score / df[df.season == 2019]['round'].unique().max()
    return model_score

In [106]:
model_score = score_classification(model)

comparison_dict['model'].append('gpr')
comparison_dict['score'].append(model_score)

display(comparison_dict)

{'model': ['gpr'], 'params': [], 'score': [3.6190656379510866]}

In [114]:
# Run the test on Bahrain data
circuits = df[df.season == 2022]['round'].unique()

circuit_ids = {
'circuit_id_adelaide' : 'circuit_id_adelaide',
'circuit_id_albert_park' : 'Australian',
'circuit_id_americas' : 'UnitedStates',
'circuit_id_bahrain' : 'Bahrain',
'circuit_id_baku' : 'Azerbaijan',
'circuit_id_brands_hatch' : 'circuit_id_brands_hatch',
'circuit_id_catalunya' : 'Spanish',
'circuit_id_detroit' : 'circuit_id_detroit',
'circuit_id_estoril' : 'circuit_id_estoril',
'circuit_id_galvez' : 'circuit_id_galvez',
'circuit_id_hockenheimring' : 'circuit_id_hockenheimring',
'circuit_id_hungaroring' : 'Hungarian',
'circuit_id_imola' : 'EmiliaRomagna',
'circuit_id_indianapolis' : 'circuit_id_indianapolis',
'circuit_id_interlagos' : 'Brazilian',
'circuit_id_istanbul' : 'circuit_id_istanbul',
'circuit_id_jacarepagua' : 'circuit_id_jacarepagua',
'circuit_id_jerez' : 'circuit_id_jerez',
'circuit_id_kyalami' : 'circuit_id_kyalami',
'circuit_id_magny_cours' : 'circuit_id_magny_cours',
'circuit_id_marina_bay' : 'Singapore',
'circuit_id_monaco' : 'Monaco',
'circuit_id_monza' : 'Italian',
'circuit_id_nurburgring' : 'circuit_id_nurburgring',
'circuit_id_phoenix' : 'circuit_id_phoenix',
'circuit_id_red_bull_ring' : 'Austrian',
'circuit_id_ricard' : 'French',
'circuit_id_rodriguez' : 'MexicoCity',
'circuit_id_sepang' : 'circuit_id_sepang',
'circuit_id_shanghai' : 'circuit_id_shanghai',
'circuit_id_silverstone' : 'British',
'circuit_id_sochi' : 'circuit_id_sochi',
'circuit_id_spa' : 'Belgian',
'circuit_id_suzuka' : 'Japanese',
'circuit_id_valencia' : 'circuit_id_valencia',
'circuit_id_villeneuve' : 'Canadian',
'circuit_id_yas_marina' : 'AbuDhabi',
'circuit_id_yeongam' : 'circuit_id_yeongam',
'circuit_id_zandvoort' : 'Dutch'
}

driverId_dict = {
'hamilton' : 'Lewis Hamilton',
'alonso' : 'Fernando Alonso',
'vettel' : 'Sebastian Vettel',
'gasly' : 'Pierre Gasly',
'ricciardo' : 'Daniel Ricciardo',
'bottas' : 'Valtteri Bottas',
'kevin_magnussen' : 'Kevin Magnussen',
'max_verstappen' : 'Max Verstappen',
'sainz' : 'Carlos Sainz',
'ocon' : 'Esteban Ocon',
'stroll' : 'Lance Stroll',
'leclerc' : 'Charles Leclerc',
'norris' : 'Lando Norris',
'russell' : 'George Russell',
'latifi' : 'Nicholas Latifi',
'tsunoda' : 'Yuki Tsunoda',
'mick_schumacher' : 'Mick Schumacher',
'zhou' : 'Zhou Guanyu',
'albon' : 'Alexander Albon',
'perez' : 'Sergio Perez',
'hulkenberg' : 'Nico Hulkenberg'
}

num_dict = {}
for i in range(20):
    num_dict[i] = i+1

for i in range(len(circuits)):
    test_circuit = df[(df.season == 2022) & (df['round'] == circuits[i])]

    circuit_name = 'UNKNOWN'
    for key in circuit_ids.keys():
        if test_circuit[key].all():
            circuit_name = circuit_ids[key]


    X_test = test_circuit.drop(['driver', 'podium'], axis = 1)
    y_test = test_circuit.podium

    #scaling
    X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

    # make predictions
    #test = np.transpose(np.array(model.predict(X_test, return_std=True)))
    test = pd.DataFrame(model.predict_proba(X_test))

    df_420 = test
    df_420['DNF'] = test[20] + test[21] + test[22] + test[23] + test[24] + test[25] + test[26]
    df_420 = df_420.drop([20, 21, 22, 23, 24, 25, 26], axis = 1)
    df_420 = df_420.rename(columns=num_dict)

    df_420['Driver'] = test_circuit['driver'].reset_index(drop=True)
    df_420['Driver'] = df_420['Driver'].map(driverId_dict)

    df_420.to_csv('../Processed Data/Dummy Probability Outputs/Logistic Regression Test/' + circuit_name + '.csv')


{0: 1,
 1: 2,
 2: 3,
 3: 4,
 4: 5,
 5: 6,
 6: 7,
 7: 8,
 8: 9,
 9: 10,
 10: 11,
 11: 12,
 12: 13,
 13: 14,
 14: 15,
 15: 16,
 16: 17,
 17: 18,
 18: 19,
 19: 20}